# 1. Libraries

In [1]:
# Base ------------------------------------------------
import numpy as np
import pandas as pd
import scipy.stats as ss
import statsmodels.stats.weightstats as smw
import math
import plotly.express as px

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.1f}'.format

#plt.style.use('seaborn')
# Viz -------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

# Correlation libraries -------------------------------------
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

# ML --------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import itertools

c:\Users\Carlos_de_Olaguibel\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Carlos_de_Olaguibel\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\Carlos_de_Olaguibel\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Carlos_de_Olaguibel\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 2. Functions

In [2]:
def model_xgb(X, y, X_train, X_test, y_train, y_test):
       
    # Define the columns to be one-hot encoded and scaled
    categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
    numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
    
    # Define the transformers for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            #('num', StandardScaler(), numerical_cols),
            ('num', RobustScaler(), numerical_cols),
            ('cat', ce.OneHotEncoder(), categorical_cols)
        ])
    
    # XGBoost Regressor
    xgb_regressor = Pipeline(steps=[('preprocessor', preprocessor),
                                        ('model', xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, n_jobs= -1))])
    xgb_regressor.fit(X_train, y_train)
    xgb_pred = xgb_regressor.predict(X_test)
    xgb_r2 = r2_score(y_test, xgb_pred)
    xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
    
    # Create a DataFrame to hold the results
    results_df = pd.DataFrame({
        'Model': ['XGBoost'],
        'R-squared': [xgb_r2],
        'RMSE': [xgb_rmse]
    })
    
    return results_df, xgb_pred

# 3. Dataset

In [3]:
# MAIN DATA SET
df = pd.read_csv('IE_Intrum_Challenge_data_clean.csv')
df.head(3)

,latitude,longitude,new_flag,price,total_building_floors,elevator_flag,construction_year,condominium_fees,ownership,floor,typology,visibility,Region,surface_res,surface_comm,price_per_m2,Heating_Adj,Air_Conditioning_Adj,Condition,room_Adj,Property_Class,bathroom_Adj,energy_efficiency_Adj
0,37.6,15.0,0.0,310000,3.0,0,1975.0,0.0,Full ownership,0.0,Single-family villa,premium,Sicilia,200.0,NaN,1550.0,heating,No AC,Renovated,5+,Mid-range,3,Unknown
1,37.6,15.1,0.0,329000,2.0,0,1895.0,0.0,Full ownership,0.0,Single-family villa,premium,Sicilia,300.0,NaN,1096.7,No heating,No AC,Needs renovation,5+,Luxury,2,G
2,37.5,15.1,0.0,330000,2.0,0,2000.0,0.0,Full ownership,0.0,Single-family villa,premium,Sicilia,148.0,174.5,2229.7,heating,AC,Renovated,4,Mid-range,3,E


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192696 entries, 0 to 192695
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   latitude               192696 non-null  float64
 1   longitude              192696 non-null  float64
 2   new_flag               192696 non-null  float64
 3   price                  192696 non-null  int64  
 4   total_building_floors  192696 non-null  float64
 5   elevator_flag          192696 non-null  int64  
 6   construction_year      192696 non-null  float64
 7   condominium_fees       192696 non-null  float64
 8   ownership              192696 non-null  object 
 9   floor                  192696 non-null  float64
 10  typology               192696 non-null  object 
 11  visibility             192696 non-null  object 
 12  Region                 192696 non-null  object 
 13  surface_res            192696 non-null  float64
 14  surface_comm           17111 non-nul

# 4. Data Cleaning

In [5]:
# Drop the columns that are not needed for the model
df_model = df.drop(['surface_comm', 'price_per_m2', 'latitude', 'longitude', 'visibility'], axis=1)

# Convert new_flag, elevator_flag and construction_year to categorical
df_model['new_flag'] = df_model['new_flag'].astype('object')
df_model['elevator_flag'] = df_model['elevator_flag'].astype('object')

# Create a column for years since construction
df['years_of_construction'] = 2023 - df['construction_year']
df_model['years_of_construction'] = 2023 - df_model['construction_year']
df_model.drop(['construction_year'], axis=1, inplace=True)

Exclude Outliers on price

In [6]:
# Calculate the lower and upper limits
grouped = df_model.groupby('Region')
lower_limit = grouped['price'].transform(lambda x: x.quantile(0.25) - 1.5 * (x.quantile(0.75) - x.quantile(0.25)))
upper_limit = grouped['price'].transform(lambda x: x.quantile(0.75) + 1.5 * (x.quantile(0.75) - x.quantile(0.25)))

# Filter the DataFrame
df_model = df_model.loc[(df_model['price'] >= lower_limit) & (df_model['price'] <= upper_limit)]


In [7]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180136 entries, 0 to 192695
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   new_flag               180136 non-null  object 
 1   price                  180136 non-null  int64  
 2   total_building_floors  180136 non-null  float64
 3   elevator_flag          180136 non-null  object 
 4   condominium_fees       180136 non-null  float64
 5   ownership              180136 non-null  object 
 6   floor                  180136 non-null  float64
 7   typology               180136 non-null  object 
 8   Region                 180136 non-null  object 
 9   surface_res            180136 non-null  float64
 10  Heating_Adj            180136 non-null  object 
 11  Air_Conditioning_Adj   180136 non-null  object 
 12  Condition              180136 non-null  object 
 13  room_Adj               180136 non-null  object 
 14  Property_Class         180136 non-nu

In [8]:
# Media of price and count of records per Region. Sort  by count format with 2 decimals
df_model.groupby('Region').agg({'price': ['mean', 'median','min','max','count']}).sort_values(('price', 'count'), ascending=False).applymap('{:,.2f}'.format)
#df_model['Region'].value_counts()

price                                              
                      mean      median        min         max      count
Region                                                                  
Lazio           249,224.90  220,000.00   5,000.00  675,000.00  37,808.00
Piemonte        150,031.26  129,000.00   3,500.00  444,000.00  28,545.00
Lombardia       274,997.25  245,000.00   6,000.00  749,000.00  26,048.00
Sicilia         139,045.30  125,000.00   8,000.00  380,000.00  13,176.00
Veneto          243,474.36  228,000.00  10,000.00  640,000.00  12,718.00
Toscana         335,523.13  290,000.00  30,000.00  940,000.00  11,390.00
Campania        223,540.55  200,000.00   5,000.00  630,000.00   9,638.00
Emilia-Romagna  253,794.00  229,000.00  11,000.00  689,000.00   8,919.00
Calabria        103,442.17   89,000.00   5,000.00  285,000.00   7,958.00
Sardegna        188,819.80  159,000.00   5,000.00  560,000.00   7,908.00
Liguria         191,152.34  150,000.00   4,000.00  645,000.00   7,013.00
Puglia          163,651.19  149,000.00   4,000.00  445,000.00   6,675.00
Abruzzo         168,975.16  150,000.00  15,000.00  450,000.00   2,340.00

# 5. Model Run

Print the results of the model

In [133]:
# create a list of regions
regions = df_model['Region'].unique().tolist()

# Regional loop
for region in regions:
    df_model_region = df_model[df_model['Region'] == region]
    X = df_model_region.drop(['price'], axis=1)
    y = df_model_region['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    print('Region: ', region, X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    results, xgb_pred = model_xgb(X, y, X_train, X_test, y_train, y_test)
    print(results)

Region:  Sicilia (13176, 17) (13176,) (9882, 17) (3294, 17) (9882,) (3294,)
     Model  R-squared    RMSE
0  XGBoost        0.6 49983.7
Region:  Lazio (37808, 17) (37808,) (28356, 17) (9452, 17) (28356,) (9452,)
     Model  R-squared    RMSE
0  XGBoost        0.7 77041.1
Region:  Piemonte (28545, 17) (28545,) (21408, 17) (7137, 17) (21408,) (7137,)
     Model  R-squared    RMSE
0  XGBoost        0.7 54666.3
Region:  Veneto (12718, 17) (12718,) (9538, 17) (3180, 17) (9538,) (3180,)
     Model  R-squared    RMSE
0  XGBoost        0.6 80295.2
Region:  Lombardia (26048, 17) (26048,) (19536, 17) (6512, 17) (19536,) (6512,)
     Model  R-squared    RMSE
0  XGBoost        0.6 93515.9
Region:  Campania (9638, 17) (9638,) (7228, 17) (2410, 17) (7228,) (2410,)
     Model  R-squared    RMSE
0  XGBoost        0.6 85996.1
Region:  Liguria (7013, 17) (7013,) (5259, 17) (1754, 17) (5259,) (1754,)
     Model  R-squared    RMSE
0  XGBoost        0.5 96465.7
Region:  Toscana (11390, 17) (11390,) (8542, 

# 6. Save results to a dataframe

In [9]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# create a list of regions
regions = df_model['Region'].unique().tolist()
# regions = ['Abruzzo', 'Lazio']
# Regional loop
df_results = pd.DataFrame()
for region in regions:
    df_model_region = df_model[df_model['Region'] == region]
    X = df_model_region.drop(['price'], axis=1)
    y = df_model_region['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    print('Region: ', region, X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    results, xgb_pred = model_xgb(X, y, X_train, X_test, y_train, y_test)
    # Create dataframes for training, testing, and prediction results
    df_train = pd.DataFrame(X_train, columns=X.columns)
    df_train['price'] = y_train

    df_test = pd.DataFrame(X_test, columns=X.columns)
    df_test['price'] = y_test
    df_test['Predicted'] = xgb_pred

    # Concatenate the dataframes
    df_combined = pd.concat([df_train, df_test])
    df_results = df_results.append(df_combined)
    print(results)

Region:  Sicilia (13176, 17) (13176,) (9882, 17) (3294, 17) (9882,) (3294,)
     Model  R-squared    RMSE
0  XGBoost        0.6 49983.7
Region:  Lazio (37808, 17) (37808,) (28356, 17) (9452, 17) (28356,) (9452,)
     Model  R-squared    RMSE
0  XGBoost        0.7 77041.1
Region:  Piemonte (28545, 17) (28545,) (21408, 17) (7137, 17) (21408,) (7137,)
     Model  R-squared    RMSE
0  XGBoost        0.7 54666.3
Region:  Veneto (12718, 17) (12718,) (9538, 17) (3180, 17) (9538,) (3180,)
     Model  R-squared    RMSE
0  XGBoost        0.6 80295.2
Region:  Lombardia (26048, 17) (26048,) (19536, 17) (6512, 17) (19536,) (6512,)
     Model  R-squared    RMSE
0  XGBoost        0.6 93515.9
Region:  Campania (9638, 17) (9638,) (7228, 17) (2410, 17) (7228,) (2410,)
     Model  R-squared    RMSE
0  XGBoost        0.6 85996.1
Region:  Liguria (7013, 17) (7013,) (5259, 17) (1754, 17) (5259,) (1754,)
     Model  R-squared    RMSE
0  XGBoost        0.5 96465.7
Region:  Toscana (11390, 17) (11390,) (8542, 

In [10]:
df_results.describe()

,total_building_floors,condominium_fees,floor,surface_res,years_of_construction,price,Predicted
count,180136.0,180136.0,180136.0,180136.0,180136.0,180136.0,45038.0
mean,3.4,41.8,1.4,118.3,52.5,217510.2,217471.1
std,1.9,68.7,1.7,79.7,54.3,141989.5,119780.8
min,1.0,0.0,0.0,20.0,0.0,3500.0,-32200.4
25%,2.0,0.0,0.0,72.0,32.0,114000.0,126781.7
50%,3.0,1.0,1.0,100.0,51.0,183000.0,193261.0
75%,5.0,62.0,2.0,136.0,63.0,290000.0,284444.1
max,10.0,993.0,60.0,1000.0,1023.0,940000.0,840275.4


In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Assuming 'df' is your dataframe with columns 'price' and 'predicted'

filtered_df = df_results.dropna(subset=['Predicted'])
n = filtered_df.shape[0]
p = 1

r2 = r2_score(filtered_df['price'], filtered_df['Predicted'])
adj_r2 = adjusted_r2(r2, n, p)
rmse = np.sqrt(mean_squared_error(filtered_df['price'], filtered_df['Predicted']))

print("Adjusted R^2:", adj_r2)
print("R^2:", r2)
print("RMSE:", rmse)


Adjusted R^2: 0.7038925908565296
R^2: 0.7038991656152644
RMSE: 77204.49176393198


# 7. Save results to a csv file

In [172]:
df_results.to_csv('Intrum_Prediction.csv', index=False)